 1. Imports

In [15]:
import os
import sys
import json
import duckdb
import pandas as pd

# Set project root path for module imports
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from memosynth.vector_store import write_memory, query_memory, create_collection
from memosynth.memory_client import diff, resolve, create_collection, init_timeline_table, load_memory_and_insert

Memory [1] inserted into Qdrant!
Memory [1] logged to DuckDB!
Memory [1] added to Neo4j graph!
Memory [8a86f314-bf15-4327-a08b-7be7d2a6ff11] inserted into Qdrant!
Memory [8a86f314-bf15-4327-a08b-7be7d2a6ff11] logged to DuckDB!
Memory [8a86f314-bf15-4327-a08b-7be7d2a6ff11] added to Neo4j graph!


2. Load Memory JSON File

In [2]:
# Load a memory JSON object
memory_path = "/Users/samhi/Downloads/memosynth-lite/config/sample_memory.json"

with open(memory_path, "r") as f:
    memory = json.load(f)

print(" Sample memory loaded:")
print(json.dumps(memory, indent=2))


 Sample memory loaded:
[
  {
    "id": 1,
    "content": "Today I learned about vector databases and Qdrant.",
    "project": "demo_project",
    "agent": "doc_bot",
    "summary": "Client asked about margin drop in Q2.",
    "type": "insight",
    "tags": [
      "finance",
      "Q2",
      "risk"
    ],
    "source": "Earnings_Report_Q2.pdf",
    "author": "doc_bot",
    "created_at": "2025-06-19",
    "version": 1,
    "confidence": 0.9,
    "visibility": "project",
    "sensitivity": "medium",
    "timestamp": "2025-06-19T12:00:00Z"
  },
  {
    "id": "8a86f314-bf15-4327-a08b-7be7d2a6ff11",
    "summary": "User asked about fixing the Neo4j connection error.",
    "source": "testing",
    "timestamp": "2025-06-21T17:30:00Z"
  }
]


3. Create Qdrant Collection and Insert Memory

In [5]:
create_collection()
print(" Qdrant collection created or already exists.")

# Loop through each memory item (because your JSON is a list)
required_keys = ["id", "summary", "created_at", "content"]

for mem in memory:
    if all(k in mem for k in required_keys):
        write_memory(mem)
        print(f" Memory written to Qdrant: {mem['id']}")
    else:
        print(f" Skipped incomplete memory: {mem.get('id', 'unknown')} (Missing keys)")


 Qdrant collection created or already exists.
 Memory written to Qdrant: 1
 Skipped incomplete memory: 8a86f314-bf15-4327-a08b-7be7d2a6ff11 (Missing keys)


4. Query Memory from Qdrant

In [7]:
# Query Qdrant with a prompt
query_text = "Tell me about vector databases"
results = query_memory(query_text)

print(f"\n Top matches for: '{query_text}'\n")
for idx, mem in enumerate(results, 1):
    print(f"{idx}. Summary: {mem.get('summary', 'No summary')}")
    print(f"   ➤ Project: {mem.get('project', 'N/A')}")
    print(f"   ➤ Created At: {mem.get('created_at', 'N/A')}")
    print(f"   ➤ Tags: {', '.join(mem.get('tags', []))}\n")


 Top matches for: 'Tell me about vector databases'

1. Summary: User asked about fixing the Neo4j connection error.
   ➤ Project: N/A
   ➤ Created At: N/A
   ➤ Tags: 

2. Summary: User asked about fixing the Neo4j connection error.
   ➤ Project: N/A
   ➤ Created At: N/A
   ➤ Tags: 

3. Summary: User asked about fixing the Neo4j connection error.
   ➤ Project: N/A
   ➤ Created At: N/A
   ➤ Tags: 



5. Insert Memory into DuckDB

In [9]:
# Convert the memory to a DataFrame
df = pd.DataFrame([memory])

# Connect to DuckDB file
con = duckdb.connect("memory_log.duckdb")

# Register DataFrame as a view
con.register("df_view", df)

# Create table and insert data (if not already exists)
con.execute("""
    CREATE TABLE IF NOT EXISTS memory_log AS
    SELECT * FROM df_view
""")

print("Memory inserted into DuckDB.")

Memory inserted into DuckDB.


 6. Query from DuckDB and Display

In [11]:
# Simple query: Get key info from memory_log
result_df = con.execute("SELECT summary, project, created_at FROM memory_log").df()

print(" DuckDB Stored Memories:")
print(result_df)


 DuckDB Stored Memories:
                                 summary       project  created_at
0  Client asked about margin drop in Q2.  demo_project  2025-06-19


7. Cypher Query to Inspect Neo4j Memory Nodes

In [13]:
from memosynth.graph_store import driver

# Run a test query to return all Memory nodes
with driver.session() as session:
    results = session.run("MATCH (m:Memory) RETURN m.id AS id, m.content AS content, m.timestamp AS timestamp")
    print(" Memory nodes in Neo4j:\n")
    for record in results:
        print(f"ID: {record['id']}")
        print(f"Content: {record['content']}")
        print(f"Timestamp: {record['timestamp']}")
        print("-" * 40)

 Memory nodes in Neo4j:

ID: 1
Content: Today I learned about vector databases and Qdrant.
Timestamp: 2025-06-19T12:00:00Z
----------------------------------------
ID: 1
Content: Today I learned about vector databases and Qdrant.
Timestamp: 2025-06-19T12:00:00Z
----------------------------------------
ID: 1
Content: Today I learned about vector databases and Qdrant.
Timestamp: 2025-06-19T12:00:00Z
----------------------------------------
ID: test1
Content: N/A
Timestamp: 2025-06-21T17:30:00Z
----------------------------------------
ID: 1
Content: Today I learned about vector databases and Qdrant.
Timestamp: 2025-06-19T12:00:00Z
----------------------------------------
ID: test1
Content: N/A
Timestamp: 2025-06-21T17:30:00Z
----------------------------------------
ID: 1
Content: Today I learned about vector databases and Qdrant.
Timestamp: 2025-06-19T12:00:00Z
----------------------------------------
ID: test1
Content: N/A
Timestamp: 2025-06-21T17:30:00Z
---------------------------------

In [17]:
from memosynth.memory_client import diff, resolve, create_collection, init_timeline_table, load_memory_and_insert

mem1 = {"summary": "Client asked about margin drop in Q2."}
mem2 = {"summary": "Client was worried about infrastructure costs in Q2."}

print(diff(mem1, mem2))
print(resolve(mem1, mem2))

if __name__ == "__main__":
    create_collection()
    init_timeline_table()
    load_memory_and_insert()


 Difference in summaries:
1: Client asked about margin drop in Q2.
2: Client was worried about infrastructure costs in Q2.
🤖 Resolved Summary: The client was concerned about Q2 margin and infrastructure costs.
Memory [1] inserted into Qdrant!
Memory [1] logged to DuckDB!
Memory [1] added to Neo4j graph!
Memory [8a86f314-bf15-4327-a08b-7be7d2a6ff11] inserted into Qdrant!
Memory [8a86f314-bf15-4327-a08b-7be7d2a6ff11] logged to DuckDB!
Memory [8a86f314-bf15-4327-a08b-7be7d2a6ff11] added to Neo4j graph!
